In [ ]:
from os.path import join, basename
import os
import numpy as np
import pandas as pd
import xarray as xr
# spatial libraries
import geopandas as gp 

from  shapely.geometry import Point
import rasterio
import rasterio.transform
# flow direction library
import pyflwdir # https://gitlab.com/deltares/wflow/pyflwdir

In [ ]:
# function to read values from spatial raster
def sample_map(x, y, fn_map, layer=1):
    """read map values at x, y coordinates from file"""
    if not os.path.isfile(fn_map): 
        raise IOError("{} file not found".format(fn_map))
    with rasterio.open(fn_map, 'r') as src:
        # assume low resolution lat lon coordinates are given
        r, c = src.index(x, y)
        r, c = np.atleast_1d(r).astype(int), np.atleast_1d(c).astype(int)
        nrows, ncols = src.shape
        valid = np.logical_and.reduce((r>=0, r<nrows, c>=0, c<ncols))
        sample = np.ones(r.size, dtype=src.dtypes[layer-1])*np.nan
        sample[valid] = src.read(layer)[r[valid], c[valid]]
    return sample

def sample_map_mem(x, y, map_data, transform):
    """read map_data (np.ndarray) values at x, y coordinates"""
    r, c = rasterio.transform.rowcol(transform, x, y)
    r, c = np.atleast_1d(r).astype(int), np.atleast_1d(c).astype(int)
    nrows, ncols = map_data.shape
    valid = np.logical_and.reduce((r>=0, r<nrows, c>=0, c<ncols))
    sample = np.ones(r.size, dtype=map_data.dtype)*np.nan
    sample[valid] = map_data[r[valid], c[valid]]
    return sample

In [ ]:
# read some cama-flood maps and parse flow direction data

map_dir = r'/home/dirk/models/cama-flood_bmi_v3.6.2_nc/map/global_15min'

fn_nextxy = join(map_dir, 'nextxy.tif')
with rasterio.open(fn_nextxy, 'r') as src:
    nextxy = src.read()
    transform = src.transform
    flw = pyflwdir.FlwdirRaster(data=nextxy, ftype='flow')


In [ ]:
# accumulated ucat worldpop data over flow direction
fn_exp_agg = join(map_dir, 'worldpop.tif')
with rasterio.open(fn_exp_agg, 'r') as src:
    exp_agg = src.read(1)
    with rasterio.open(fn_exp_agg.replace('.tif', '_accu.tif'), 'w', **src.profile) as dst:
        exp_agg_accu = flw.accuflux(exp_agg)
        dst.write(exp_agg_accu.astype(np.float32), 1)

fn_exp_agg_lecz = join(map_dir, 'lecz_worldpop.tif')
with rasterio.open(fn_exp_agg_lecz, 'r') as src:
    exp_agg_lecz = src.read(1)     
    with rasterio.open(fn_exp_agg_lecz.replace('.tif', '_accu.tif'), 'w', **src.profile) as dst:
        exp_agg_lecz_accu = flw.accuflux(exp_agg_lecz)
        dst.write(exp_agg_lecz_accu.astype(np.float32), 1)

In [ ]:
# combine simulations
ddir = join(root, 'data', '3-model_output')
scens = ['cmpnd', 'runoff']
mods = ['anu', 'cnrs', 'ecmwf', 'jrc', 'nerc']
rp=10
mod_lst = []
for mod in mods:
    scen_lst = []
    for scen in scens:
        fn_swe = join(ddir, f'ev_map_sfcelv_{mod}_mswep_{scen}_v362_1980-2014.nc')
        scen_lst.append(xr.open_dataset(fn_swe, chunks={'lat':360, 'lon':360})['sfcelv_ev'])
    mod_lst.append(xr.concat(scen_lst, dim='scen'))
ds = xr.concat(mod_lst, dim='ensemble')
ds['ensemble'] = xr.Variable('ensemble', mods)
ds['scen'] = xr.Variable('scen', scens)
ds_diff = (ds.sel(T=rp, scen='cmpnd')-ds.sel(T=rp, scen='runoff')).compute()

In [ ]:
# calculated people affected by dH

import warnings
warnings.simplefilter('ignore')
coupling = pd.read_csv(fn_csv_coupling2, index_col='index')
lon, lat, index = coupling['cmf_lon_15min'], coupling['cmf_lat_15min'], coupling.index

# read total exposure
fn_exp_agg = join(map_dir, 'worldpop_accu.tif')
fn_exp_agg_lecz = join(map_dir, 'lecz_worldpop_accu.tif')
pop_all_rivmth = sample_map(lon, lat, fn_exp_agg)
pop_lecz_rivmth = sample_map(lon, lat, fn_exp_agg_lecz)


rps =[2, 10, 50, 100]
mod_lst = []
for mod in mods:
    rp_lst =[]
    for rp in rps:
        fn_tif = join(ddir, f'{mod}_cmpnd', f'worldpop_agg_T{rp:03d}.tif')
        
        with rasterio.open(fn_tif, 'r') as src:
            mv_mask = src.read(1)==src.nodata
            
            # accumulate total no. of people affected
            pop_affected_all_accu = flw.accuflux(src.read(1))
            pop_affected_all_accu[mv_mask] = src.nodata
            pop_affected_all_rivmth = sample_map_mem(lon, lat, pop_affected_all_accu, src.transform)
                
            # in flood plains with larger flood depth
            ds_diff = (ds.sel(T=rp, scen='cmpnd')-ds.sel(T=rp, scen='runoff')).compute()
            mask = ds_diff.sel(ensemble=mod).squeeze().values>0.01   
            pop_affected_dH = np.where(mask, src.read(1), 0)
            pop_affected_dH_accu = flw.accuflux(pop_affected_dH)
            pop_affected_dH_accu[mv_mask] = src.nodata
            pop_affected_dH_rivmth = sample_map_mem(lon, lat, pop_affected_dH_accu, src.transform)

            rp_lst.append(xr.merge([
                xr.DataArray(dims=('index'), data=pop_affected_dH_rivmth, coords=dict(index=index), name='people_affected_dH'),
                xr.DataArray(dims=('index'), data=pop_affected_all_rivmth, coords=dict(index=index), name='people_affected_all'),
            ]))
    mod_lst.append(xr.concat(rp_lst, dim='T'))
ds_out = xr.merge([
    xr.concat(mod_lst, dim='ensemble'),
    xr.DataArray(dims=('index'), data=pop_all_rivmth, coords=dict(index=index), name='people_all'),
    xr.DataArray(dims=('index'), data=pop_lecz_rivmth, coords=dict(index=index), name='people_lecz'),
])
ds_out['ensemble'] = xr.Variable('ensemble', mods)
ds_out['T'] = xr.Variable('T', rps)
ds_out.to_netcdf(join(ddir, 'rivmth_pop_affected.nc'))